#**Smart Wardrobe: Integrating YOLOv8 and Large Language Models in Fashion Styling.**

###1. Install YOLOv8 and Import YOLO and other Dependencies

In [4]:
!pip install ultralytics
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import numpy as np
import yaml
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import os
from PIL import Image
import requests
from io import BytesIO
import torch
import openai
from openai import OpenAI
import requests
from io import BytesIO
from google.colab import files

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


###2. Model Setup

In [6]:
#Load the model
model = YOLO("yolov8n-cls.yaml")

YOLOv8n-cls summary: 99 layers, 2,719,288 parameters, 2,719,288 gradients, 4.4 GFLOPs


###3. Training and testing the Model

During training, YOLO internally validates the model. This process ensures that the model isn't just memorizing the training data but is generalizing well to unseen examples. This process will also download the FashionMnist dataset.

In [7]:
#Train the model
results = model.train(data="fashion-mnist", epochs=10, imgsz=28)

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.yaml, data=fashion-mnist, epochs=10, time=None, patience=100, batch=16, imgsz=28, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

100%|██████████| 47.0M/47.0M [00:00<00:00, 159MB/s]
Unzipping /content/datasets/fashion-mnist.zip to /content/datasets/fashion-mnist...: 100%|██████████| 70023/70023 [00:07<00:00, 9430.05file/s]

Dataset download success ✅ (9.4s), saved to /content/datasets/fashion-mnist



train: /content/datasets/fashion-mnist/train... found 60000 images in 10 classes ✅ 
val: None...
test: /content/datasets/fashion-mnist/test... found 10000 images in 10 classes ✅ 
Overriding model.yaml nc=1000 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3,

train: Scanning /content/datasets/fashion-mnist/train... 60000 images, 0 corrupt: 100%|██████████| 60000/60000 [00:16<00:00, 3656.59it/s]


...

      10/10         0G     0.5207         16         32: 100%|██████████| 3750/3750 [06:26<00:00,  9.70it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 313/313 [00:16<00:00, 19.06it/s]

                   all       0.88      0.999



10 epochs completed in 1.169 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (Intel Xeon 2.00GHz)
YOLOv8n-cls summary (fused): 73 layers, 1,447,690 parameters, 0 gradients, 3.3 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/datasets/fashion-mnist/train... found 60000 images in 10 classes ✅ 
val: None...
test: /content/datasets/fashion-mnist/test... found 10000 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 313/313 [00:15<00:00, 20.10it/s]


                   all       0.88      0.999
Speed: 0.0ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


### Running Inference on the model Using some samples from the test data
Defined the class labels for Fashion MNIST

In [8]:
# Define the manual class labels for Fashion MNIST
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"
]

In [9]:
# Inference for a specific image
results = model.predict(source="/content/datasets/fashion-mnist/test/9/1007.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()

        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/9/1007.png: 32x32 9 0.95, 7 0.05, 5 0.00, 8 0.00, 3 0.00, 2.7ms
Speed: 4.7ms preprocess, 2.7ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train2
Predicted class: Ankle Boot, Confidence: 0.95


In [10]:
# Inference for a more images
results = model.predict(source="/content/datasets/fashion-mnist/test/0/1049.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/0/1049.png: 32x32 0 0.93, 6 0.06, 3 0.00, 2 0.00, 4 0.00, 2.7ms
Speed: 1.2ms preprocess, 2.7ms inference, 0.1ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train3
Predicted class: T-shirt/top, Confidence: 0.93


In [11]:
# Inference for a more images
results = model.predict(source="/content/datasets/fashion-mnist/test/7/1040.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/7/1040.png: 32x32 7 1.00, 5 0.00, 9 0.00, 8 0.00, 0 0.00, 2.6ms
Speed: 1.0ms preprocess, 2.6ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train4
Predicted class: Sneaker, Confidence: 1.00


In [12]:
# Inference for a more images
results = model.predict(source="/content/datasets/fashion-mnist/test/8/1564.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/8/1564.png: 32x32 8 1.00, 4 0.00, 0 0.00, 6 0.00, 3 0.00, 2.7ms
Speed: 1.0ms preprocess, 2.7ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train5
Predicted class: Bag, Confidence: 1.00


### Testing on Google Images

In [13]:
# Using an image from the web
image_url = "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?q=80&w=1780&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Download the image from the URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Preprocessing the image for YOLO
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1))
])
img_tensor = transform(img).unsqueeze(0)

# Perform inference
results = model.predict(source=img_tensor, save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        predicted_class_idx = result.probs.top1
        predicted_class_name = class_names[predicted_class_idx]
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



0: 640x640 6 0.31, 8 0.17, 0 0.12, 4 0.11, 2 0.11, 35.3ms
Speed: 0.0ms preprocess, 35.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/train6
Predicted class: Shirt, Confidence: 0.31


# **STYLING ADVICE PORTION**
###**PARSING THE IMAGE TO AN LLM TO GIVE STYLING ADVICE**
The LLM chosen is OpenAI 's ChatGPT

###Testing my API and GPT 4's advice to see if the advice is suitable for the user

In [17]:
# Set your API key
client =OpenAI(api_key="INSERT-API-KEY")

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
      {'role': 'user',
       'content': 'You are a fashion expert. Suggest stylish outfit combinations for someone wearing an ankle boot.'}
  ]
)

message = response.choices[0].message.content
print(message)


1. Casual Chic: A pair of ankle boots with skinny jeans, a plain white t-shirt and a lightweight cardigan. Accessorise with a chunky necklace or scarf for some added flare.

2. Boho Vibe: Pair the ankle boots with a flowy maxi dress and a wide brimmed hat. This style works great for summer festivals or outdoor parties.

3. Business Casual: Wear your ankle boots with a knee-length pencil skirt and a fitted blouse. This look would be appropriate for a work environment while still being fashionable and comfortable. A structured jacket can be added for an extra polished finish.

4. Edgy Look: Black or metallic ankle boots combined with leather pants and a band t-shirt can make for a chic, edgy outfit. Top it off with a bomber jacket for a cool, casual look. 

5. Elegant Ensemble: Try wearing your ankle boots with a midi dress and a sleek, leather jacket for an evening out. Add a splash of color to the ensemble with a bright clutch or statement jewelry.

6. Cozy Winter Outfit: Ankle boots l

### Trying to test the code to give fashion advice on a google image after the url has been inputed.

In [15]:
# Using an image from the web
image_url = "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?q=80&w=1780&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Download the image from the URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Preprocessing the image for YOLO
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1))
])
img_tensor = transform(img).unsqueeze(0)

# Perform inference
results = model.predict(source=img_tensor, save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        predicted_class_idx = result.probs.top1
        predicted_class_name = class_names[predicted_class_idx]
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")

# Set your API key
client =OpenAI(api_key="INSERT-API-KEY")

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {'role': 'user',
         'content': f'You are a fashion expert. Suggest stylish outfit combinations for someone wearing a {predicted_class_name}.'}
    ]
)

message = response.choices[0].message.content
print(message)




0: 640x640 6 0.31, 8 0.17, 0 0.12, 4 0.11, 2 0.11, 24.0ms
Speed: 0.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/train7
Predicted class: Shirt, Confidence: 0.31
1. Classic combination: A shirt with a well-fitted suit and tie, paired with a pair of brogues or loafers. Add some elegance with a pocket square and a sleek watch. 

2. Casual Day Outfit: A plaid or patterned shirt paired with slim-fit jeans and white sneakers. Top up the look with a cap or minimalist watch for a relaxing meetup.

3. Office Attire: A crisp white shirt with navy or black trousers. Pair it with oxford shoes or derby shoes and a leather belt. Add a tie for a formal meeting.

4. Street Style: A denim shirt layered over a basic t-shirt, paired with skinny jeans or ripped jeans and high-top sneakers. Finish the look with a snapback cap and a backpack. 

5. Summer Vibes: A printed short-sleeve shirt with cotton shorts and espadrilles or sandals

### Giving fashion advice after the user uploads their own image
Will be uploading the image of a bag i found online

In [19]:
# Step 1: Upload the image
uploaded = files.upload()

Saving a bag.jpg to a bag.jpg


In [20]:
# Get the uploaded file path
image_path = list(uploaded.keys())[0]

# Open the uploaded image
img = Image.open(image_path)

# Step 2: Preprocessing the image for YOLO
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1))
])

img_tensor = transform(img).unsqueeze(0)

# Step 3: YOLO Prediction (Assuming 'model' is defined already)
results = model.predict(source=img_tensor, save=True)

# Step 4: Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        predicted_class_idx = result.probs.top1
        predicted_class_name = class_names[predicted_class_idx]
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")

# Step 5: Ask OpenAI for fashion advice based on the predicted class
client =OpenAI(api_key="INSERT-API-KEY")


response = client.chat.completions.create(
    model="gpt-4",
    messages=[{
        'role': 'user',
        'content': f'You are a fashion expert. Suggest stylish outfit combinations for someone wearing a {predicted_class_name}.'}]
)

message = response.choices[0].message.content
print(message)


0: 640x640 8 0.45, 6 0.18, 0 0.14, 3 0.07, 2 0.06, 22.2ms
Speed: 0.0ms preprocess, 22.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/train8
Predicted class: Bag, Confidence: 0.45
1. **Casual Chic**
   - Bag: Leather tote bag in a neutral color
   - Outfit: High waist skinny jeans, simple white t-shirt
   - Shoes: Loafers or white sneakers
   - Accessories: Gold hoop earrings, sunglasses 

2. **Boho Outfit**
   - Bag: Fringe crossbody bag
   - Outfit: Flowy chic maxi dress in floral prints
   - Shoes: Gladiator sandals
   - Accessories: Wide-brimmed hat, layered necklaces 

3. **Business Casual**
   - Bag: Structured handbag in a bold color
   - Outfit: Black pencil skirt, button-down white blouse
   - Shoes: Black pointed heels
   - Accessories: Thin silver watch, stud earrings  

4. **Street Style**
   - Bag: Small backpack in bright or pastel colors
   - Outfit: Ripped jeans, oversized graphic tee or crop top
   - Shoes: Chunky sn